## Aim

Write a general ABC sampler, to understand the technique and to apply it to exoplanet occurrence problems.

In [1]:
import numpy as np
import scipy
import scipy.stats as stats
import functools

In [9]:
class ABCSampler:
    def __init__(self, prior, candidate_getter, distance=lambda a, b: abs(a - b), statistic=np.mean):
        '''
        prior : stats.rv_continuous or stats.rv_discrete object.
        Samples from the prior distribution over the parameters.
        Takes in nothing.
        Returns numpy.ndarray of parameters.

        candidate_getter : function
        Gets candidate samplers of the posterior distribution.
        Takes in parameters, as returned by prior.
        Returns function 'candidate' that samples from the candidate distribution, 
        to be compared to true data, which
            Takes in the number of sample points to return.
            Returns numpy.ndarray of sample points of the desired shape.
        '''
        self.prior = prior
        self.candidate_getter = candidate_getter
        self.distance = distance
        self.statistic = statistic
        
    def sample(self, data, prior=None, max_iters=float('inf'), threshold=1e-1):
        '''
        data : numpy.ndarray
        Data that we want to fit. 
        Arbitrary shape, but a row should match the return type of candidate.

        max_iters : scalar
        The number of times to try and accept a candidate.

        threshold : scalar
        The maximum value of distance allowable to accept candidate.
        '''
        num_iters = 0
        if prior is None:
            prior = self.prior
        while True:
            params = prior.rvs()
            candidate = self.candidate_getter(params)
            synthetic = candidate(data.shape[0])
            dis = self.distance(self.statistic(synthetic), self.statistic(data))
            if dis <= threshold:
                return params
            num_iters += 1
            if num_iters > max_iters:
                return None
            
    def sample_pmc(self, data, thresholds, num_walkers):
        '''
        Carries out population Monte Carlo ABC, based on Beaumont et al. (2009).
        https://arxiv.org/pdf/0805.2256.pdf
        '''
        sampler = functools.partial(self.sample, data=data, max_iters=float('inf'))
        params_matrix = np.array([sampler(threshold=thresholds[0]) for _ in range(num_walkers)])
        weights = np.ones((num_walkers,)) / num_walkers
        tau = 2 * np.cov(params_matrix.T)
        for thresh in thresholds[1:]:
            new_params_matrix = np.empty_like(params_matrix)
            new_weights = np.empty_like(weights)
            for i in range(num_walkers):
                center = params_matrix[np.random.choice(num_walkers, p=weights)]
                sampling_normal = stats.multivariate_normal(center, tau)
                param_i = sampler(prior=sampling_normal, threshold=thresh)
                new_params_matrix[i] = param_i
                new_weights[i] = prior.pdf(param_i) / np.dot(weights, np.prod(stats.norm.pdf(
                    np.linalg.inv(scipy.linalg.sqrtm(tau)).dot((param_i - params_matrix).T)), axis=0)) # is the sqrtm needed?
            params_matrix = new_params_matrix
            weights = new_weights / sum(new_weights)
        return np.mean(params_matrix, axis=0) # average the final results

In [10]:
# First simple example: a Gaussian fit you can do with MLE.
# Suppose we've sampled data from N(50, 10), but our best guess is it's N(N(40, 20), N(11, 2)).
prior = stats.multivariate_normal(np.array([40, 11]), np.diag([20, 2]))
def candidate_getter(p):
    def candidate(size):
        return np.random.normal(p[0], np.abs(p[1]), size)
    return candidate

gaussian_sampler = ABCSampler(prior, candidate_getter)
gaussian_sampler.sample(np.random.normal(50, 10, (100,)))

array([49.99957201, 11.34914444])

In [11]:
# and now, PMC!
# let's do the same exact problem as in the first one
gaussian_sampler.sample_pmc(np.random.normal(50, 10, (100,)), [0.5, 0.1, 0.01, 0.001], 10)

array([48.77746338, 11.90057682])

In [12]:
# Exp(22) but we estimate Exp(max(0, N(20, 2)))
prior = stats.halfnorm(20, 2)
def candidate_getter(p):
    def candidate(size):
        return np.random.exponential(p, size)
    return candidate

exp_sampler = ABCSampler(prior, candidate_getter)
exp_sampler.sample(np.random.exponential(22, (100,)), threshold=0.001)

20.12162930203327

1. https://arxiv.org/pdf/1802.09720.pdf